<a href="https://colab.research.google.com/github/Leonid393/projects/blob/main/tweets_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Классификация на эмбеддингах

In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
df_tweets = pd.read_csv('/datasets/tweets.csv')

In [ ]:
df_tweets.sample(n = 400)

,text,positive
2181,RT @Insecteced: Буклеты #лабмедиа на конференц...,1
2689,День просто супер:( http://t.co/crlqAuLBY6,0
3097,Расплакалась над танцем Алианы и её отца ( так...,0
2265,"@Ju1ia15 @initialness я добавил, надеюсь помож...",1
2073,"Написал киткату, пробили у курьеров информацию...",1
...,...,...
233,Буду смотреть концерн Земфиры в прямом эфире) ...,1
671,2 дня *О* \nP.s нагло тырю фотки с инсты х) ht...,1
4183,"@UFO_or_I не печалься, Симон! Ты де позитив! ...",0
1330,RT @DariaClever: я теперь немного звизда\nменя...,1


In [ ]:
tokenizer = transformers.BertTokenizer(
    vocab_file='/datasets/ds_bert/vocab.txt')

tokenized = df_tweets['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
config = transformers.BertConfig.from_json_file(
    '/datasets/ds_bert/bert_config.json')
model = transformers.BertModel.from_pretrained(
    '/datasets/ds_bert/rubert_model.bin', config=config)

In [ ]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

In [ ]:
features = np.concatenate(embeddings)

x_train, x_test = train_test_split(features, shuffle=None, test_size = 0.5)

y = df_tweets['positive']
y_trian, y_test = train_test_split(y, shuffle=None, test_size=0.5)

lr = LogisticRegression(solver='liblinear').fix(x_train, y_trian)
prediction = lr.predict(x_test)
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, prediction))
# обучите и протестируйте модель
# < напишите код здесь >